In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import re
import datetime
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import norm, probplot
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from arch.unitroot import VarianceRatio
import scipy.stats as stats
from statsmodels.tsa.stattools import adfuller 
from scipy.stats import genpareto
from sklearn.utils import resample
from scipy.optimize import minimize
from scipy.special import expit, logit
from sklearn.linear_model import LogisticRegression
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from scipy.optimize import minimize
from scipy.stats import genpareto
from scipy.optimize import differential_evolution
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from scipy.optimize import differential_evolution
from scipy.stats import gumbel_r
from statsmodels.graphics.tsaplots import plot_acf
import ta

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

### Import Data

In [77]:
btc = pd.read_csv(r'/Users/xiaoyuxu/Desktop/Dissertation-docts/bitcoin_2017_to_2023.csv').iloc[::-1]
btc.set_index(btc.columns[0], inplace=True)
btc = btc[~btc.index.duplicated(keep='first')]
btc.index = pd.to_datetime(btc.index)

btc_min_return = pd.read_csv('btc_min_return_selected.csv')
btc_min_return.index =pd.to_datetime(btc_min_return['timestamp'])
btc_min_return.drop(columns=['timestamp'],inplace=True)

append_features=btc.columns.to_list()
append_features.remove('close')
append_features.remove('volume')

btc_min_return=pd.merge(btc[append_features], btc_min_return, left_index=True, right_index=True, how='inner')
btc_min_return

,open,high,low,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,close,return,volume,...,left_threshold,right_threshold,shape_left,scale_left,shape_right,scale_right,VaR_left,VaR_right,VaR_left_general,VaR_right_general
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-09-11 14:01:00,4260.41,4260.41,4260.41,0.000000e+00,0,0.00000,0.000000,4260.41,0.000000,0.000000,...,0.042757,0.043907,0.072527,0.026335,0.101222,0.024352,0.005815,0.005953,0.005288,0.005338
2017-09-11 14:02:00,4260.52,4260.52,4260.52,1.335503e+03,1,0.00000,0.000000,4260.52,0.000026,0.313460,...,0.042757,0.043907,0.072527,0.026335,0.101222,0.024352,0.005815,0.005953,0.005288,0.005338
2017-09-11 14:03:00,4260.52,4260.52,4260.52,0.000000e+00,0,0.00000,0.000000,4260.52,0.000000,0.000000,...,0.042757,0.043907,0.072527,0.026335,0.101222,0.024352,0.005815,0.005954,0.005289,0.005338
2017-09-11 14:04:00,4265.02,4265.02,4265.02,6.688234e+02,1,0.00000,0.000000,4265.02,0.001056,0.156816,...,0.042757,0.043907,0.072527,0.026335,0.101222,0.024352,0.005815,0.005954,0.005289,0.005338
2017-09-11 14:05:00,4289.15,4289.16,4289.15,7.720486e+03,2,1.80000,7720.485506,4289.16,0.005644,1.800000,...,0.042757,0.043907,0.072527,0.026335,0.101222,0.024352,0.005815,0.005954,0.005289,0.005338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 13:15:00,28896.00,28907.42,28893.03,1.090761e+06,686,16.50452,476955.246611,28907.41,0.000395,37.746570,...,0.039136,0.044071,0.517513,0.016578,1.290333,0.010263,0.000583,0.000650,0.000585,0.000590
2023-08-01 13:16:00,28907.41,28912.74,28907.41,4.595556e+05,483,10.22981,295738.166916,28908.53,0.000039,15.896100,...,0.039136,0.044071,0.517513,0.016578,1.290333,0.010263,0.000583,0.000650,0.000585,0.000590
2023-08-01 13:17:00,28908.52,28908.53,28902.48,3.330532e+05,451,2.70873,78290.170121,28902.49,-0.000209,11.522630,...,0.039136,0.044071,0.517513,0.016578,1.290333,0.010263,0.000583,0.000650,0.000585,0.000590


### Generate Basic Features

In [80]:
# Create rolling statistics
btc_min_return['rolling_mean_return_5min'] = btc_min_return['return'].rolling(window=5).mean()
btc_min_return['rolling_std_return_5min'] = btc_min_return['return'].rolling(window=5).std()
btc_min_return['rolling_mean_volume_5min'] = btc_min_return['volume'].rolling(window=5).mean()
btc_min_return['rolling_std_volume_5min'] = btc_min_return['volume'].rolling(window=5).std()

# Create lag features (1, 5, 10, 15, 30 minutes)
lags = [1, 5, 10, 15, 30]
for lag in lags:
    btc_min_return[f'return_lag_{lag}min'] = btc_min_return['return'].shift(lag)
    btc_min_return[f'volume_lag_{lag}min'] = btc_min_return['volume'].shift(lag)

# Interaction features
btc_min_return['return_volume_interaction'] = btc_min_return['return'] * btc_min_return['volume']

# Exponential moving averages
btc_min_return['ema_return_5min'] = btc_min_return['return'].ewm(span=5, adjust=False).mean()

# Volume-based indicators
btc_min_return['vwap'] = (btc_min_return['return'] * btc_min_return['volume']).cumsum() / btc_min_return['volume'].cumsum()

# Volatility measures
btc_min_return['historical_volatility_5min'] = btc_min_return['return'].rolling(window=5).std()

# Momentum indicators
btc_min_return['roc_5min'] = btc_min_return['return'].pct_change(periods=5)

# Technical Indicators 
# Moving Average Convergence Divergence (MACD)
btc_min_return['macd'] = ta.trend.MACD(btc_min_return['return']).macd()

# Relative Strength Index (RSI)
btc_min_return['rsi'] = ta.momentum.RSIIndicator(btc_min_return['return']).rsi()

# Bollinger Bands
bollinger = ta.volatility.BollingerBands(btc_min_return['return'])
btc_min_return['bollinger_hband'] = bollinger.bollinger_hband()
btc_min_return['bollinger_lband'] = bollinger.bollinger_lband()

# Average True Range (ATR)
atr = ta.volatility.AverageTrueRange(
    high=btc_min_return['close'], 
    low=btc_min_return['low'], 
    close=btc_min_return['close']
)
btc_min_return['atr'] = atr.average_true_range()

btc_min_return=btc_min_return.dropna()

### Feature Engeering

### Benchmark Linear Regression Model and Feature Selection

In [ ]:
features = btc_min_return.columns.to_list()
df = btc_min_return.copy()
df = df.dropna(subset=features, how='all')
df['target'] = df['return'].shift(-1)
X = df[features]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")